In [1]:
!pwd

/Users/Pavlo/PycharmProjects/ADL-Minicar-Challenge-2023/mycar/notebooks


In [1]:
import os
import time
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.2.0


In [2]:
!which python

/Users/Pavlo/miniconda3/envs/donkey/bin/python


In [16]:
ls ../models

arrow_sign_classifier/         pedestrian_detector_v5.tflite
arrow_sign_classifier.tflite   pedestrian_detector_v7/
car_detector/                  pedestrian_detector_v7.tflite
car_detector.tflite            pedestrian_detector_v8/
car_detector_v3/               pedestrian_detector_v8.tflite
pedestrian_detector_v2.0.zip   stop_sign_detector/
pedestrian_detector_v4/        stop_sign_detector.tflite
pedestrian_detector_v4.tflite  stop_sign_detector_v8/
pedestrian_detector_v5/        stop_sign_detector_v8.tflite


In [26]:
model_path = "../models/stop_sign_detector_v9/"
tflite_model_path = "../models/stop_sign_detector_v9.tflite"

In [4]:
# Workaround for this bug https://github.com/tensorflow/tensorflow/issues/34698
bin_path = os.path.dirname(os.sys.executable)
if 'PATH' in os.environ:
    os.environ['PATH'] += ':' + bin_path
else:
    os.environ['PATH'] = bin_path
print(bin_path)
print(os.environ['PATH'])

/Users/Pavlo/opt/anaconda3/envs/donkey/bin
/Users/Pavlo/opt/anaconda3/envs/donkey/bin:/Users/Pavlo/opt/anaconda3/bin:/Users/Pavlo/opt/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/Library/Apple/usr/bin:/usr/local/Cellar/sumo/1.14.1.reinstall/share/sumo/bin:/Users/Pavlo/opt/anaconda3/bin:/Users/Pavlo/opt/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/Library/Apple/usr/bin:/usr/local/Cellar/sumo/1.14.1.reinstall/share/sumo/bin:/Users/Pavlo/opt/anaconda3/envs/donkey/bin


In [27]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

2023-01-24 18:42:25.488434: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2023-01-24 18:42:25.488794: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2023-01-24 18:42:25.494490: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:797] Optimization results for grappler item: graph_to_optimize
2023-01-24 18:42:25.494505: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:799]   function_optimizer: Graph size after: 61 nodes (48), 83 edges (70), time = 2.75ms.
2023-01-24 18:42:25.494509: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:799]   function_optimizer: function_optimizer did nothing. time = 0.126ms.
2023-01-24 18:42:25.577262: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2023-01-24 18:42:25.

In [28]:
model = tf.keras.models.load_model(model_path)

input_data = np.array(np.random.random_sample([1, 70, 160, 3]), dtype=np.float32)
print(input_data.shape)

start = time.perf_counter()

output_data = model(input_data)

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

(1, 70, 160, 3)
tf.Tensor([[0.99947554]], shape=(1, 1), dtype=float32)
Elapsed Time: 0.008009543000298436 ms


In [29]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print(input_data.shape)

# Inference
start = time.perf_counter()

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

(1, 70, 160, 3)
[[0.9995098]]
Elapsed Time: 0.006382753999787383 ms
